# FrequencyStorm - Variable Duration Storms

This notebook demonstrates how to generate HEC-HMS "Frequency Storm" hyetographs
for different storm durations (6-hour, 12-hour, 24-hour, 48-hour, etc.).

## Key Concepts

The FrequencyStorm module provides two methods:

1. **Alternating Block Method** (`generate_alternating_block`): 
   - Standard HMS method for Frequency Storms
   - Supports ANY duration
   - Duration determined by the longest duration in your DDF table

2. **M3 Lookup Method** (`generate_hyetograph`):
   - Extracted from HCFCD M3 ground truth
   - 24-hour storms ONLY
   - Perfect match to HCFCD M3 model output

## References

- [HMS Frequency Storm Documentation](https://www.hec.usace.army.mil/confluence/hmsdocs/hmstrm/meteorology/precipitation/frequency-storm)
- [HMS Hypothetical Storm Documentation](https://www.hec.usace.army.mil/confluence/hmsdocs/hmstrm/meteorology/precipitation/hypothetical-storm)

In [ ]:
# pip install hms-commander

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from hms_commander import FrequencyStorm

print(f"FrequencyStorm module loaded")

## Example DDF Data (TP-40 Houston Area)

For Frequency Storms, you need cumulative precipitation depths at various durations.
The storm duration equals the **longest duration** in your table.

| Duration | Depth (1% AEP) |
|----------|----------------|
| 5 min    | 1.20"          |
| 15 min   | 2.10"          |
| 30 min   | 3.18"          |
| 60 min   | 4.30"          |
| 2 hr     | 5.70"          |
| 3 hr     | 6.80"          |
| 6 hr     | 9.10"          |
| 12 hr    | 11.10"         |
| 24 hr    | 13.20"         |
| 48 hr    | 16.20"         |
| 4 day    | 18.50"         |
| 7 day    | 21.00"         |
| 10 day   | 23.50"         |

In [ ]:
# Complete DDF data (1% AEP, Houston area example)
# To create a storm of a specific duration, only include depths UP TO that duration

FULL_DEPTHS = [1.20, 2.10, 3.18, 4.30, 5.70, 6.80, 9.10, 11.10, 13.20, 16.20, 18.50, 21.00, 23.50]
FULL_DURATIONS = [5, 15, 30, 60, 120, 180, 360, 720, 1440, 2880, 5760, 10080, 14400]  # minutes

# Duration names for reference
DURATION_NAMES = ['5min', '15min', '30min', '1hr', '2hr', '3hr', '6hr', '12hr', '24hr', '48hr', '4day', '7day', '10day']

## 6-Hour Storm

For a 6-hour storm, provide depths up to 360 minutes (6 hours).

In [ ]:
# 6-hour storm - depths up to index 6 (360 min)
depths_6hr = FULL_DEPTHS[:7]      # [5min, 15min, 30min, 1hr, 2hr, 3hr, 6hr]
durations_6hr = FULL_DURATIONS[:7]  # [5, 15, 30, 60, 120, 180, 360]

print(f"6-hour storm DDF:")
for dur, depth, name in zip(durations_6hr, depths_6hr, DURATION_NAMES[:7]):
    print(f"  {name:>6}: {depth:.2f} inches")

# Generate hyetograph
hyeto_6hr = FrequencyStorm.generate_alternating_block(
    depths_6hr, durations_6hr, 
    time_interval_min=5, 
    peak_position_pct=67
)

print(f"\nGenerated hyetograph:")
print(f"  Intervals: {len(hyeto_6hr)} (6hr / 5min = 72)")
print(f"  Total: {hyeto_6hr.sum():.2f} inches")
print(f"  Peak: {hyeto_6hr.max():.3f} inches")
print(f"  Peak at: {(np.argmax(hyeto_6hr)+1)*5/60:.1f} hours (67% of 6hr = 4.0hr)")

## 12-Hour Storm

In [ ]:
# 12-hour storm - depths up to index 7 (720 min)
depths_12hr = FULL_DEPTHS[:8]
durations_12hr = FULL_DURATIONS[:8]

hyeto_12hr = FrequencyStorm.generate_alternating_block(
    depths_12hr, durations_12hr,
    time_interval_min=5,
    peak_position_pct=67
)

print(f"12-hour storm:")
print(f"  Intervals: {len(hyeto_12hr)} (12hr / 5min = 144)")
print(f"  Total: {hyeto_12hr.sum():.2f} inches")
print(f"  Peak at: {(np.argmax(hyeto_12hr)+1)*5/60:.1f} hours")

## 24-Hour Storm (Two Methods)

For 24-hour storms, you can use either:
1. **Alternating block** - standard HMS method
2. **M3 lookup** - matches HCFCD M3 model output exactly

In [ ]:
# 24-hour storm - depths up to index 8 (1440 min)
depths_24hr = FULL_DEPTHS[:9]
durations_24hr = FULL_DURATIONS[:9]

# Method 1: Alternating block
hyeto_24hr_ab = FrequencyStorm.generate_alternating_block(
    depths_24hr, durations_24hr,
    time_interval_min=5,
    peak_position_pct=67
)

# Method 2: M3 lookup (HCFCD compatible)
hyeto_24hr_m3 = FrequencyStorm.generate_hyetograph(
    total_depth=13.20,
    time_interval_min=5,
    peak_position_pct=67
)

print(f"24-hour storm comparison:")
print(f"\nAlternating Block:")
print(f"  Total: {hyeto_24hr_ab.sum():.2f} inches")
print(f"  Peak: {hyeto_24hr_ab.max():.3f} inches")

print(f"\nM3 Lookup (HCFCD):")
print(f"  Total: {hyeto_24hr_m3.sum():.2f} inches")
print(f"  Peak: {hyeto_24hr_m3.max():.3f} inches")

# Compare the two methods
diff = hyeto_24hr_ab - hyeto_24hr_m3
print(f"\nDifference:")
print(f"  Max absolute diff: {np.abs(diff).max():.4f} inches")
print(f"  RMSE: {np.sqrt(np.mean(diff**2)):.4f} inches")

## 48-Hour Storm

In [ ]:
# 48-hour storm - depths up to index 9 (2880 min)
depths_48hr = FULL_DEPTHS[:10]
durations_48hr = FULL_DURATIONS[:10]

hyeto_48hr = FrequencyStorm.generate_alternating_block(
    depths_48hr, durations_48hr,
    time_interval_min=5,
    peak_position_pct=67
)

print(f"48-hour storm:")
print(f"  Intervals: {len(hyeto_48hr)} (48hr / 5min = 576)")
print(f"  Total: {hyeto_48hr.sum():.2f} inches")
print(f"  Peak at: {(np.argmax(hyeto_48hr)+1)*5/60:.1f} hours")

## 7-Day Storm (Extended Duration)

In [ ]:
# 7-day storm - depths up to index 11 (10080 min)
depths_7day = FULL_DEPTHS[:12]
durations_7day = FULL_DURATIONS[:12]

# Use 15-minute intervals for longer storms to reduce array size
hyeto_7day = FrequencyStorm.generate_alternating_block(
    depths_7day, durations_7day,
    time_interval_min=15,  # 15-min intervals
    peak_position_pct=67
)

print(f"7-day storm:")
print(f"  Intervals: {len(hyeto_7day)} (7day / 15min = 672)")
print(f"  Total: {hyeto_7day.sum():.2f} inches")
print(f"  Peak at: {(np.argmax(hyeto_7day)+1)*15/60:.1f} hours ({(np.argmax(hyeto_7day)+1)*15/60/24:.1f} days)")

## Visual Comparison of Storm Durations

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 6-hour
ax = axes[0, 0]
time_6hr = np.arange(len(hyeto_6hr)) * 5 / 60  # hours
ax.bar(time_6hr, hyeto_6hr, width=5/60*0.8, color='steelblue', edgecolor='navy')
ax.axvline(x=6*0.67, color='red', linestyle='--', label='Peak (67%)')
ax.set_xlabel('Time (hours)')
ax.set_ylabel('Precipitation (inches)')
ax.set_title(f'6-Hour Storm\nTotal: {hyeto_6hr.sum():.2f}", Peak: {hyeto_6hr.max():.2f}"')
ax.legend()
ax.grid(True, alpha=0.3)

# 12-hour
ax = axes[0, 1]
time_12hr = np.arange(len(hyeto_12hr)) * 5 / 60
ax.bar(time_12hr, hyeto_12hr, width=5/60*0.8, color='steelblue', edgecolor='navy')
ax.axvline(x=12*0.67, color='red', linestyle='--', label='Peak (67%)')
ax.set_xlabel('Time (hours)')
ax.set_ylabel('Precipitation (inches)')
ax.set_title(f'12-Hour Storm\nTotal: {hyeto_12hr.sum():.2f}", Peak: {hyeto_12hr.max():.2f}"')
ax.legend()
ax.grid(True, alpha=0.3)

# 24-hour comparison
ax = axes[1, 0]
time_24hr = np.arange(len(hyeto_24hr_ab)) * 5 / 60
ax.bar(time_24hr - 0.02, hyeto_24hr_ab, width=5/60*0.4, color='steelblue', 
       edgecolor='navy', label='Alternating Block', alpha=0.7)
ax.bar(time_24hr + 0.02, hyeto_24hr_m3, width=5/60*0.4, color='coral', 
       edgecolor='darkred', label='M3 Lookup', alpha=0.7)
ax.axvline(x=24*0.67, color='red', linestyle='--', label='Peak (67%)')
ax.set_xlabel('Time (hours)')
ax.set_ylabel('Precipitation (inches)')
ax.set_title(f'24-Hour Storm Comparison\nTotal: {hyeto_24hr_ab.sum():.2f}"')
ax.legend()
ax.grid(True, alpha=0.3)

# 48-hour
ax = axes[1, 1]
time_48hr = np.arange(len(hyeto_48hr)) * 5 / 60
ax.bar(time_48hr, hyeto_48hr, width=5/60*0.8, color='steelblue', edgecolor='navy')
ax.axvline(x=48*0.67, color='red', linestyle='--', label='Peak (67%)')
ax.set_xlabel('Time (hours)')
ax.set_ylabel('Precipitation (inches)')
ax.set_title(f'48-Hour Storm\nTotal: {hyeto_48hr.sum():.2f}", Peak: {hyeto_48hr.max():.2f}"')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('frequency_storm_durations.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved: frequency_storm_durations.png")

## Peak Position Options

HMS supports 5 standard peak positions: 25%, 33%, 50%, 67%, and 75%.

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(18, 4))

peak_positions = [25, 33, 50, 67, 75]
time_24hr = np.arange(288) * 5 / 60  # hours

for ax, pct in zip(axes, peak_positions):
    hyeto = FrequencyStorm.generate_alternating_block(
        depths_24hr, durations_24hr,
        time_interval_min=5,
        peak_position_pct=pct
    )
    
    ax.bar(time_24hr, hyeto, width=5/60*0.8, color='steelblue', edgecolor='navy')
    ax.axvline(x=24*pct/100, color='red', linestyle='--', linewidth=2)
    ax.set_xlabel('Time (hours)')
    ax.set_ylabel('Precip (in)')
    ax.set_title(f'Peak at {pct}%\n({24*pct/100:.0f} hours)')
    ax.set_xlim(0, 24)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('frequency_storm_peak_positions.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved: frequency_storm_peak_positions.png")

## Using generate_from_ddf() Convenience Method

The `generate_from_ddf()` method provides a convenient interface with a `method` parameter.

In [ ]:
# Using generate_from_ddf with alternating_block method (default)
hyeto_ddf_ab = FrequencyStorm.generate_from_ddf(
    depths_6hr, durations_6hr,
    peak_position_pct=67,
    time_interval_min=5,
    method='alternating_block'
)
print(f"6-hr via generate_from_ddf (alternating_block): {hyeto_ddf_ab.sum():.2f} inches")

# Using generate_from_ddf with m3_lookup method (24-hour only)
hyeto_ddf_m3 = FrequencyStorm.generate_from_ddf(
    depths_24hr, durations_24hr,
    peak_position_pct=67,
    time_interval_min=5,
    method='m3_lookup'
)
print(f"24-hr via generate_from_ddf (m3_lookup): {hyeto_ddf_m3.sum():.2f} inches")

## Summary

| Duration | Method | Intervals (5-min) | Example Total Depth |
|----------|--------|-------------------|--------------------|
| 6-hour   | alternating_block | 72 | 9.10" |
| 12-hour  | alternating_block | 144 | 11.10" |
| 24-hour  | alternating_block or m3_lookup | 288 | 13.20" |
| 48-hour  | alternating_block | 576 | 16.20" |
| 4-day    | alternating_block | 1152 | 18.50" |
| 7-day    | alternating_block | 2016 | 21.00" |
| 10-day   | alternating_block | 2880 | 23.50" |

**Key Points:**
- Storm duration = longest duration in your DDF table
- For HCFCD M3 compatibility (24-hr only), use `generate_hyetograph()` or `method='m3_lookup'`
- For other durations, use `generate_alternating_block()` or `method='alternating_block'`
- Peak positions: 25%, 33%, 50%, 67%, 75%
- HMS supports durations up to 10 days